# Use the spatio-temporal library for location analytics


This notebook shows you how to work with the spatio-temporal library that is pre-installed on all Spark environments in Watson Studio. You can use the spatio-temporal library to expand your data science analysis in Python notebooks to include location analytics by gathering, manipulating, and displaying imagery, GPS, satellite photography and historical data.

In this notebook, you will learn how to get started with using the library, how to read and write data using `pyst` which supports most of the common geospatial formats, which includes shapefile, GeoJSON and Well-Known Text (WKT). You will learn how to use topological relations to confine the returned results of your location data analysis, which geohashing functions to use for proximity search (encoding latitude and longitude and grouping nearby points), and how to calculate the distance between points using ellipsoidal metrics. 

This notebook runs on the latest Python and Spark.


## Table of contents
1. [Create a spatial context](#createContext)
1. [Read and write data](#io)<br>
1. [Topological functions](#topo)<br>
1. [Geohashing functions](#geohash)<br>
1. [Ellipsoidal metrics](#elli)<br>
1. [Summary](#summary)


<a id="createContext"></a>
## Create a spatial context

Before you can start using any of the spatio-temporal library functions in your notebook, you must register `STContext` to access the `st` functions. The `STContext` is linked to the Spark session.

To register `STContext`:

In [1]:
from pyst import STContext
# Register STContext, which is the main entry point
stc = STContext(spark.sparkContext._gateway)

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20220218222108-0000
KERNEL_ID = f94a450b-c878-460a-8fa7-d19a5d2f9da2


<a name="io"></a> 
## Read and write data

Watson Studio supports most of the common geospatial formats, including shapefile, GeoJSON and Well-Known Text (WKT).

Each data reader function can take various types of input to its `.read()` method. For example, a URL pointing to a resource file, a path to a local GeoJSON file, streaming bytes from a GeoJSON file, or content in GeoJSON format from a Python dictionary.

In this notebook, you will learn how to work with data in GeoJSON and WKT format. 

### geoJSON support

To work with files in GeoJSON format, first create a GeoJSON reader and writer. 

The `geojson_reader` returns a pandas DataFrame when calling the `.read()` method. And the returned pandas DataFrame has one column called `geometry` that contains the spatial objects of the geometries while the remaining columns contain the properties of geometries.

In [2]:
geojson_reader = stc.geojson_reader()
geojson_writer = stc.geojson_writer()

The following code snippets show reading US counties polygon data from various kinds of sources.

- Read a GeoJSON file on your local machine:

In [5]:
! wget -q https://api.dataplatform.cloud.ibm.com/v2/gallery-assets/entries/1ec43d48a694c6c1d052ddca4d68bdc4/data?accessKey=842f098fa70cd5ea4e7cfecc1b1ae759

In [6]:
df = geojson_reader.read('counties.geojson')
df.head(3)

/opt/ibm/conda/miniconda/lib/python/site-packages/pyst/io/geojson_reader.py:59: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(properties)


,AFFGEOID,STATEFP,ALAND,LSAD,GEOID,COUNTYFP,COUNTYNS,AWATER,NAME,geometry
0,0500000US37017,37,2265887723,06,37017,017,01026336,33010866,Bladen,MultiPolygon(Polygon: Boundary: Ring(LineSegme...
1,0500000US37167,37,1023370459,06,37167,167,01025844,25242751,Stanly,MultiPolygon(Polygon: Boundary: Ring(LineSegme...
2,0500000US39153,39,1069181981,06,39153,153,01074088,18958267,Summit,MultiPolygon(Polygon: Boundary: Ring(LineSegme...


- Read a Python dictionary that contains the GeoJSON content:

In [7]:
import json

In [8]:
data = json.load(open('counties.geojson'))

In [9]:
df = geojson_reader.read(data)
df.head(3)

/opt/ibm/conda/miniconda/lib/python/site-packages/pyst/io/geojson_reader.py:59: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(properties)


,AFFGEOID,STATEFP,ALAND,LSAD,GEOID,COUNTYFP,COUNTYNS,AWATER,NAME,geometry
0,0500000US37017,37,2265887723,06,37017,017,01026336,33010866,Bladen,MultiPolygon(Polygon: Boundary: Ring(LineSegme...
1,0500000US37167,37,1023370459,06,37167,167,01025844,25242751,Stanly,MultiPolygon(Polygon: Boundary: Ring(LineSegme...
2,0500000US39153,39,1069181981,06,39153,153,01074088,18958267,Summit,MultiPolygon(Polygon: Boundary: Ring(LineSegme...


- Read data directly from an securely encrypted URL that points to a GeoJSON file:

In [8]:
#df = geojson_reader.read('https://api.dataplatform.cloud.ibm.com/v2/gallery-assets/entries/1ec43d48a694c6c1d052ddca4d68bdc4/data?accessKey=1ec43d48a694c6c1d052ddca4d69054c')
#df.head(3)

,AFFGEOID,ALAND,AWATER,COUNTYFP,COUNTYNS,GEOID,LSAD,NAME,STATEFP,geometry
0,0500000US37017,2265887723,33010866,017,01026336,37017,06,Bladen,37,MultiPolygon(Polygon: Boundary: Ring(LineSegme...
1,0500000US37167,1023370459,25242751,167,01025844,37167,06,Stanly,37,MultiPolygon(Polygon: Boundary: Ring(LineSegme...
2,0500000US39153,1069181981,18958267,153,01074088,39153,06,Summit,39,MultiPolygon(Polygon: Boundary: Ring(LineSegme...


- Read binary data:

In [10]:
df = geojson_reader.read(open('counties.geojson', 'rb').read())
df.head(3)

/opt/ibm/conda/miniconda/lib/python/site-packages/pyst/io/geojson_reader.py:59: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(properties)


,AFFGEOID,STATEFP,ALAND,LSAD,GEOID,COUNTYFP,COUNTYNS,AWATER,NAME,geometry
0,0500000US37017,37,2265887723,06,37017,017,01026336,33010866,Bladen,MultiPolygon(Polygon: Boundary: Ring(LineSegme...
1,0500000US37167,37,1023370459,06,37167,167,01025844,25242751,Stanly,MultiPolygon(Polygon: Boundary: Ring(LineSegme...
2,0500000US39153,39,1069181981,06,39153,153,01074088,18958267,Summit,MultiPolygon(Polygon: Boundary: Ring(LineSegme...


- Read streaming bytes, for example from IBM Cloud Object Storage:

In [11]:
# @hidden cell
# This code cell is commented out as COS for each user is not public.
# If you wish to read data from your COS bucket, uncomment the code below and replace the ... with your COS object to run the read.

# Get streamingbody from your COS bucket
#streaming_body = ...
#df = geojson_reader.read(streaming_body.read())
#df.head(3)

### WKT support

To work with WKT strings in a Python notebook, first create a WKT reader and writer:

In [12]:
wkt_reader = stc.wkt_reader()
wkt_writer = stc.wkt_writer()

Read a WKT string to a geometry object and then query different values of the object:

In [13]:
westchester_WKT = 'POLYGON((-73.984 41.325,-73.948 41.33,-73.78 41.346,-73.625 41.363,-73.545 41.37,-73.541 41.368,-73.547 41.297,-73.485 41.223,-73.479 41.215,-73.479 41.211,-73.493 41.203,-73.509 41.197,-73.623 41.144,-73.628 41.143,-73.632 41.14,-73.722 41.099,-73.714 41.091,-73.701 41.073,-73.68 41.049,-73.68 41.047,-73.673 41.041,-73.672 41.038,-73.668 41.035,-73.652 41.015,-73.651 41.011,-73.656 41,-73.655 40.998,-73.656 40.995,-73.654 40.994,-73.654 40.987,-73.617 40.952,-73.618 40.946,-73.746 40.868,-73.751 40.868,-73.821 40.887,-73.826 40.886,-73.84 40.89,-73.844 40.896,-73.844 40.9,-73.85 40.903,-73.853 40.903,-73.854 40.9,-73.859 40.896,-73.909 40.911,-73.92 40.912,-73.923 40.914,-73.923 40.918,-73.901 40.979,-73.894 41.023,-73.893 41.043,-73.896 41.071,-73.894 41.137,-73.94 41.207,-73.965 41.24,-73.973 41.244,-73.975 41.247,-73.976 41.257,-73.973 41.266,-73.95 41.288,-73.966 41.296,-73.98 41.309,-73.984 41.311,-73.987 41.315,-73.987 41.322,-73.984 41.325))'
westchester = wkt_reader.read(westchester_WKT)

In [14]:
westchester.area()

1363618331.5760047

In [15]:
westchester.centroid()

Point(41.15551622739597, -73.7592843233704)

In [16]:
westchester.get_bounding_box()

BoundingBox: Lower Corner: Point(40.86800000000001, -73.987), Upper Corner: Point(41.36999999999998, -73.479)

Write a geometry object to a WKT string:

In [17]:
wkt_writer.write(westchester)

'POLYGON ((-73.984 41.325, -73.987 41.322, -73.987 41.315, -73.984 41.311, -73.98 41.309, -73.966 41.296, -73.95 41.288, -73.973 41.266, -73.976 41.257, -73.975 41.247, -73.973 41.244, -73.965 41.24, -73.94 41.207, -73.894 41.137, -73.896 41.071, -73.893 41.043, -73.894 41.023, -73.901 40.979, -73.923 40.918, -73.923 40.914, -73.92 40.912, -73.909 40.911, -73.859 40.896, -73.854 40.9, -73.853 40.903, -73.85 40.903, -73.844 40.9, -73.844 40.896, -73.84 40.89, -73.826 40.886, -73.821 40.887, -73.751 40.868, -73.746 40.868, -73.618 40.946, -73.617 40.952, -73.654 40.987, -73.654 40.994, -73.656 40.995, -73.655 40.998, -73.656 41.0, -73.651 41.011, -73.652 41.015, -73.668 41.035, -73.672 41.038, -73.673 41.041, -73.68 41.047, -73.68 41.049, -73.701 41.073, -73.714 41.091, -73.722 41.099, -73.632 41.14, -73.628 41.143, -73.623 41.144, -73.509 41.197, -73.493 41.203, -73.479 41.211, -73.479 41.215, -73.485 41.223, -73.547 41.297, -73.541 41.368, -73.545 41.37, -73.625 41.363, -73.78 41.346, 

### Direct input support

Besides reading geometries from different sources, direct input is also supported and each geometry type comes with a constructor.
  
These are the currently supported geometries types and their constructors:
<br>

 - Point: `point(lat, lon)`
 - LineSegment: `line_segment(start_point, end_point)` 
 - LineString: `line_string([point_1, point_2, ...])` and `line_string([line_segment_1, line_segment_2, ...])` 
 - Ring: `ring([point_1, point_2, ...])` and `ring([line_segment_1, line_segment_2, ...])` 
 - Polygon: `polygon(exterior_ring, [interior_ring_1, interior_ring_2, ...])` 
 - MultiGeometry: `multi_geometry(geom_1, geom_2, ...)` 
 - MultiPoint: `multi_point(point_1, point_2, ...)` 
 - MultiLineString: `multi_line_string(line_string_1, line_string_2, ...)`
 - MultiPolygon: `multi_polygon(polygon_1, polygon_2, ...)`
 - Null Geometry: `null_geometry()`
 - FullEarth: `full_earth()` 
 - BoundingBox: `bounding_box(lower_lat, lower_lon, upper_lat, upper_lon)`


Here are some examples of supported geometry types:

In [18]:
point_1 = stc.point(37.3, -74.4)
point_2 = stc.point(37.5,-74.1)
point_3 = stc.point(38.1, -74.4)
line_segment = stc.line_segment(point_1, point_2)
line_string = stc.line_string([point_1, point_2, point_3])
ring = stc.ring([point_1, point_2, point_3, point_1])
poly = stc.polygon(ring)
multi_points = stc.multi_point([point_1, point_2, point_3])

In [19]:
poly.area()

1179758985.44891

In [20]:
multi_points

MultiPoint(Point(37.5, -74.1), Point(38.1, -74.4), Point(37.3, -74.4))

In [21]:
next(multi_points)

Point(37.5, -74.1)

In [22]:
next(multi_points)

Point(38.1, -74.4)

In [23]:
next(multi_points)

Point(37.3, -74.4)

<a name="topo"></a>
## Topological functions

You can use topological relations to confine the returned results of your location data analysis.

The Python code snippets in this notebook, which show how to use the different topological functions, run on test data. Begin by preparing the test data in WKT string format and reading it:

In [24]:
westchester_WKT = 'POLYGON((-73.984 41.325,-73.948 41.33,-73.78 41.346,-73.625 41.363,-73.545 41.37,-73.541 41.368,-73.547 41.297,-73.485 41.223,-73.479 41.215,-73.479 41.211,-73.493 41.203,-73.509 41.197,-73.623 41.144,-73.628 41.143,-73.632 41.14,-73.722 41.099,-73.714 41.091,-73.701 41.073,-73.68 41.049,-73.68 41.047,-73.673 41.041,-73.672 41.038,-73.668 41.035,-73.652 41.015,-73.651 41.011,-73.656 41,-73.655 40.998,-73.656 40.995,-73.654 40.994,-73.654 40.987,-73.617 40.952,-73.618 40.946,-73.746 40.868,-73.751 40.868,-73.821 40.887,-73.826 40.886,-73.84 40.89,-73.844 40.896,-73.844 40.9,-73.85 40.903,-73.853 40.903,-73.854 40.9,-73.859 40.896,-73.909 40.911,-73.92 40.912,-73.923 40.914,-73.923 40.918,-73.901 40.979,-73.894 41.023,-73.893 41.043,-73.896 41.071,-73.894 41.137,-73.94 41.207,-73.965 41.24,-73.973 41.244,-73.975 41.247,-73.976 41.257,-73.973 41.266,-73.95 41.288,-73.966 41.296,-73.98 41.309,-73.984 41.311,-73.987 41.315,-73.987 41.322,-73.984 41.325))'
white_plains_WKT = 'POLYGON((-73.792 41.024,-73.794 41.031,-73.779 41.046,-73.78 41.049,-73.779 41.052,-73.776 41.054,-73.775 41.057,-73.767 41.058,-73.769 41.062,-73.768 41.067,-73.762 41.073,-73.759 41.074,-73.748 41.069,-73.746 41.056,-73.742 41.056,-73.74 41.053,-73.74 41.049,-73.749 41.04,-73.748 41.035,-73.739 41.034,-73.729 41.029,-73.725 41.025,-73.72 41.016,-73.717 41.015,-73.716 41.006,-73.718 41.002,-73.732 40.988,-73.732 40.985,-73.739 40.979,-73.745 40.978,-73.749 40.981,-73.749 40.986,-73.751 40.986,-73.756 40.991,-73.759 40.991,-73.76 40.993,-73.765 40.994,-73.769 40.997,-73.774 41.002,-73.775 41.006,-73.788 41.018,-73.792 41.024))'
manhattan_WKT = 'POLYGON((-74.023 40.709,-74.02 40.723,-74.018 40.725,-74.019 40.731,-74.016 40.737,-74.017 40.741,-74.014 40.755,-74.011 40.757,-74.011 40.761,-74.006 40.767,-74.006 40.769,-73.998 40.778,-73.996 40.778,-73.995 40.783,-73.991 40.784,-73.991 40.788,-73.98 40.8,-73.966 40.822,-73.964 40.823,-73.961 40.83,-73.957 40.832,-73.954 40.836,-73.951 40.845,-73.951 40.853,-73.947 40.855,-73.94 40.863,-73.936 40.87,-73.936 40.873,-73.93 40.881,-73.924 40.882,-73.919 40.879,-73.909 40.877,-73.906 40.873,-73.907 40.867,-73.912 40.86,-73.918 40.855,-73.931 40.835,-73.93 40.81,-73.925 40.803,-73.925 40.795,-73.939 40.781,-73.938 40.774,-73.968 40.741,-73.969 40.733,-73.967 40.73,-73.967 40.726,-73.969 40.717,-73.973 40.709,-73.977 40.706,-73.996 40.705,-73.999 40.701,-74.008 40.696,-74.016 40.696,-74.017 40.698,-74.019 40.698,-74.023 40.703,-74.023 40.709))'
westchester = wkt_reader.read(westchester_WKT)
white_plains = wkt_reader.read(white_plains_WKT)
manhattan = wkt_reader.read(manhattan_WKT)

### Topological relations

You can use the following topological relation functions in your notebook:

- **contains**: Returns true if the given geometry is completely contained by this geometry. `contains` returns the exact opposite of `within`.

In [25]:
westchester.contains(white_plains)

True

- **within**: Returns true if the given geometry is completely within this geometry. `within` returns the exact opposite of `contains`.

In [26]:
white_plains.within(westchester)

True

- **intersects**:  Returns true if the intersection does not result in an empty set. `intersects` is the opposite of `disjoint`.

In [27]:
westchester.intersects(manhattan)

False

- **disjoint**: Returns true if the intersection of the two geometries is an empty set. `disjoint` is the opposite of `intersects`.

In [28]:
westchester.disjoint(manhattan)

True

- **touch**: Returns true if none of the points common to both geometries intersect the interiors of both geometries. At least one geometry must be a `linestring`, `polygon`, `multilinestring`, or `multipolygon`.

In [29]:
westchester.touch(manhattan)

True

- **overlap**: Compares two geometries of the same dimension and returns true if their intersection set results in a geometry different from both but of the same dimension.

In [30]:
westchester.overlap(white_plains)

False

- **cross**: Returns true if the intersection results in a geometry whose dimension is one less than the maximum dimension of the two source geometries and the intersection set is interior to both source geometries.

In [31]:
westchester.cross(white_plains)

False

- **equality**: Compares two geometry A and B and only returns true if A contains B and B contains A.

In [32]:
westchester.equality(westchester)

True

### Topogical operations

You can use the following topological operations:

- **intersection**:  Returns the intersection of two geometries.

In [33]:
westchester.intersection(white_plains)

Polygon: Boundary: Ring(LineSegment(Point(41.031, -73.794), Point(41.024, -73.792)), LineSegment(Point(41.024, -73.792), Point(41.018, -73.788)), LineSegment(Point(41.018, -73.788), Point(41.006, -73.775)), ...) Interiors: 

In [34]:
westchester.intersection(white_plains).equality(white_plains)

True

- **union**: Returns the union of two geometries.

In [35]:
westchester.union(white_plains)

Polygon: Boundary: Ring(LineSegment(Point(40.998, -73.655), Point(41.0, -73.656)), LineSegment(Point(41.0, -73.656), Point(41.011, -73.651)), LineSegment(Point(41.011, -73.651), Point(41.015, -73.652)), ...) Interiors: 

In [36]:
westchester.union(white_plains).equality(westchester)

True

- **difference**: Returns the difference of two geometries.

In [37]:
westchester.difference(white_plains)

Polygon: Boundary: Ring(LineSegment(Point(41.018, -73.788), Point(41.024, -73.792)), LineSegment(Point(41.024, -73.792), Point(41.031, -73.794)), LineSegment(Point(41.031, -73.794), Point(41.046, -73.779)), ...) Interiors: Ring(LineSegment(Point(40.9, -73.844), Point(40.896, -73.844)), LineSegment(Point(40.896, -73.844), Point(40.89, -73.84)), LineSegment(Point(40.89, -73.84), Point(40.886, -73.826)), ...)

- **symmetric_difference**: Returns the symmetric difference of two geometries.

In [38]:
westchester.symmetric_difference(white_plains)

Polygon: Boundary: Ring(LineSegment(Point(41.056, -73.742), Point(41.053, -73.74)), LineSegment(Point(41.053, -73.74), Point(41.049, -73.74)), LineSegment(Point(41.049, -73.74), Point(41.04, -73.749)), ...) Interiors: Ring(LineSegment(Point(41.325, -73.984), Point(41.322, -73.987)), LineSegment(Point(41.322, -73.987), Point(41.315, -73.987)), LineSegment(Point(41.315, -73.987), Point(41.311, -73.984)), ...)

- **centroid**: Returns the centroid of the geometry.

In [39]:
westchester.centroid()

Point(41.15551622739597, -73.7592843233704)

- **buffer**: Encircles a geometry at a specified distance and returns a geometry that is the buffer that surrounds the source geometry.

In [40]:
westchester.buffer(10)

Polygon: Boundary: Ring(LineSegment(Point(41.143916595689475, -73.62295229110774), Point(41.14392272755672, -73.62292162793754)), LineSegment(Point(41.14392272755672, -73.62292162793754), Point(41.14393544183699, -73.6229118297507)), LineSegment(Point(41.14393544183699, -73.6229118297507), Point(41.19692141233404, -73.50894191674074)), ...) Interiors: 

In [41]:
westchester.buffer(10).contains(westchester)

True

- **get_bounding_box**: Returns the bounding box of the geometry.

In [42]:
westchester.get_bounding_box()

BoundingBox: Lower Corner: Point(40.86800000000001, -73.987), Upper Corner: Point(41.36999999999998, -73.479)

### Topological metrics

You can use the following functions for topological metrics:

- **area**: Returns the area of the geometry.

In [43]:
white_plains.area()

36349150.276403755

- **distance**: Returns the distance between two geometries.

In [44]:
white_plains.distance(manhattan)

17751.465287921335

- **get_topological_dimensionality**: Returns the topological dimensionality of the geometry.

In [45]:
white_plains.get_topological_dimensionality()

2

### Topological aggregation

You can use the following aggregation functions:

- Get the aggregated bounding box for a list of geometries:

In [46]:
white_plains_bbox = white_plains.get_bounding_box()
westchester_bbox = westchester.get_bounding_box()
manhattan_bbox = manhattan.get_bounding_box()

aggregated_bbox = white_plains_bbox.get_containing_bb(westchester_bbox).get_containing_bb(manhattan_bbox)

- Get the aggregated convex hull for a list of geometries:

In [47]:
white_plains_points = white_plains.get_exterior_ring().get_points()
westchester_points = westchester.get_exterior_ring().get_points()
manhattan_points = manhattan.get_exterior_ring().get_points()

all_points = white_plains_points + westchester_points + manhattan_points
hull = stc.convex_hull.compute_convex_hull(all_points)

<a name="geohash"></a> 
## Geohashing functions

The spatio-temporal library includes geohashing functions for proximity search (encoding latitude and longitude and grouping nearby points) in location data analysis.

Before you can begin calculating and grouping location data, create a geohash engine:

In [48]:
geohash = stc.geohash

### Geohash encoding functions

Encode a geographic location:

In [49]:
p = stc.point(37, -74)

Encode a geographic location to a string of base-32 encoded characters:

In [50]:
geohash.string_hash_encode(p)

'dqe6kpdue5btnubpm9npcd0000'

Encode a geographic location to a string of characters and letter:

In [51]:
geohash.string_hash_encode(p, precision=5)

'dqe6k'

Encode a geographic location to a BitVector:

In [52]:
geohash.number_hash_encode(p)

BitVector(01100101100110100110100101010101100110100110100101010101100110100110100101010101100110100110100101010101101100000000000000000000)

### Geohash decoding functions

Decode a base-32 encoded string to its decimal longitude or latitude coordinates:

In [53]:
geohash.string_hash_decode('dqeh4')

Point(37.265625, -74.443359375)

Decode base-32 encoded characters to reveal the geographic location:

In [54]:
bv = geohash.number_hash_encode(p)
geohash.number_hash_decode(bv)

Point(37.0, -74.0)

### Geohash neighbors functions

The geohash neighbors functions returns the neighboring geohash codes around a specified code.

Get the geohash neighbors for the given latitude, longitude and `bit_depth`:

In [55]:
geohash.get_all_neighbors(70, -40, 25)

[BitVector(0111110000001111100101010),
 BitVector(0111110000001111100101011),
 BitVector(0111110000001111100101110),
 BitVector(0111110000001111110000000),
 BitVector(0111110000001111110000001),
 BitVector(0111110000001111110000100),
 BitVector(0111110000001111110000010),
 BitVector(0111110000001111110000011),
 BitVector(0111110000001111110000110)]

Get the geohash neighbors for the given latitude, longitude and `bit_depth`, but only return results that are within the given distance:

In [56]:
geohash.get_all_neighbors(70, -40, 25, distance_precision=1000)

[BitVector(0111110000001111110000001),
 BitVector(0111110000001111110000100),
 BitVector(0111110000001111110000011),
 BitVector(0111110000001111110000110)]

Other useful geohash functions include:

- **expand**: Returns the center code and its neighbors. All 9 geohashes for a location are returned:

In [57]:
geohash.expand('ezs42')

['ezefp',
 'ezs40',
 'ezs41',
 'ezefr',
 'ezs42',
 'ezs43',
 'ezefx',
 'ezs48',
 'ezs49']

- **neighbors**: Returns 8 geohashes, excluding the given geohash itself:

In [58]:
geohash.neighbors('ezs42')

['ezefp', 'ezs40', 'ezs41', 'ezefr', 'ezs43', 'ezefx', 'ezs48', 'ezs49']

- **get_east**, **get_west**, **get_north**, **get_south**: Returns the east, west, north, or south geohash for a given geohash:

In [59]:
geohash.get_east('ezs42')

'ezs43'

- **bv**: Encode a geohash to a BitVector:

In [60]:
bv = geohash.number_hash_encode(p)
bv.truncate(25)

In [61]:
geohash.expand(bv)

[BitVector(0110010110011010011000101),
 BitVector(0110010110011010011010000),
 BitVector(0110010110011010011010001),
 BitVector(0110010110011010011000111),
 BitVector(0110010110011010011010010),
 BitVector(0110010110011010011010011),
 BitVector(0110010110011010011001101),
 BitVector(0110010110011010011011000),
 BitVector(0110010110011010011011001)]

In [62]:
geohash.neighbors(bv)

[BitVector(0110010110011010011000101),
 BitVector(0110010110011010011010000),
 BitVector(0110010110011010011010001),
 BitVector(0110010110011010011000111),
 BitVector(0110010110011010011010011),
 BitVector(0110010110011010011001101),
 BitVector(0110010110011010011011000),
 BitVector(0110010110011010011011001)]

In [63]:
geohash.get_north(bv)

BitVector(0110010110011010011011000)

### Geohash coverage

To calculate a set of geohashes that wholly covers the bounding box:

#### 1. Prepare a sample test polygon:

In [64]:
test_wkt = 'POLYGON((-73.76223024988917 41.04173285255264,-73.7749331917837 41.04121496082817,-73.78197130823878 41.02748934524744,-73.76476225519923 41.023733725449326,-73.75218805933741 41.031633228865495,-73.7558787789419 41.03752486433286,-73.76223024988917 41.04173285255264))'
poly = wkt_reader.read(test_wkt)

#### 2. To compute the geohash cover at a fixed bit depth for a given geometry:

In [65]:
cover = geohash.geohash_cover_at_bit_depth(poly, 36)

#### 3. To compute the buffered geohash cover at a fixed bit depth for a given geometry:

In [66]:
buffered_cover = stc.geohash.geohash_cover_at_bit_depth(poly, 36, distance=50)

#### 4. To compute a compact geohash cover by first computing the fixed depth cover and then compressing the cover:

In [67]:
raw_cover = geohash.geohash_cover_at_bit_depth(poly, 36)
compact_cover = stc.geohash.geohash_compression(raw_cover)

<a name="elli"></a> 
## Ellipsoidal metrics

You can use ellipsoidal metrics to calculate the distance between points.

Before you can begin using ellipsoidal metrics, create a metrics object:

In [68]:
eg_metric = stc.eg_metric

Examples of metrics you can compute:

- Compute the radians between two points using `azimuth`:

In [69]:
p1 = stc.point(47.1, -73.5)
p2 = stc.point(47.6, -72.9)
eg_metric.azimuth(p1, p2)

0.6802979449118038

- Compute the distance between two points in the units of the underlying data (typically in meters):

In [70]:
eg_metric.distance(p1, p2)

71730.66213673435

- Compute the landing point given a starting point, a heading (in radians), and the distance in the units of the underlying data (typically in meters):

In [71]:
point = p1
heading = eg_metric.azimuth(p1, p2)
distance = eg_metric.distance(p1, p2)
eg_metric.destination_point(p1, heading, distance)

Point(47.60000000001233, -72.89999999998498)

<a name="summary"></a> 

## Summary

In this notebook, you learnt how to get started with the geospatio-temporal library. You were shown the following: 

- Read and write data in GeoJSON and WKT format
- Use topological relations to confine the returned results of your location data analysis 
- Which geohashing functions to use for proximity search
- Calculate the distance between points using ellipsoidal metrics. 



### Author

**Linsong Chu**, Research Engineer at IBM Research

Copyright © 2019-2022 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>